В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [1]:
import numpy as np
import scipy as sp

from nltk.corpus import movie_reviews

Чтобы получить id-шники негативных и позитивных отзывов:

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Чтобы получить список негативных отзывов:

In [3]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

# Инструкция по выполнению

<div align="right">
<a href="#placeholder" class="btn btn-default" data-toggle="collapse">Развернуть список</a>
</div>
<div id="placeholder" class="collapse">
<ol>
    <li>Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.[ссылка](#1)</li>
    <li>Подсчитайте количество отзывов в выборке.[ссылка](#2)</li>
    <li>Подсчитайте долю класса 1 в выборке.[ссылка](#3)</li>
    <li>Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.[ссылка](#4)</li>
    <li>Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy.[ссылка](#5)</li>
    <li>Аналогично accuracy, оцените качество по ROC AUC.[ссылка](#6)</li>
    <li>Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer.[ссылка](#7)</li>
</ol>
</div>

### Задание #1 <a id='1'></a>

In [4]:
# negfeats - done in example, [3] cell.
# posfeats
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [5]:
feedbacks = np.append(negfeats, posfeats) # negative + positive list
feedbacks.shape

(2000,)

In [6]:
n, p = len(negfeats), len(posfeats)
negative, positive = np.zeros(n), np.ones(p)

classes = np.append(negative, positive) # 0 + 1 classes list
classes.shape

(2000,)

### Задание #2 <a id='2'></a>

In [7]:
with open("subm/w01s01.txt", 'w+') as f:
    f.write(str(feedbacks.shape[0]))

print(feedbacks.shape[0])

2000


### Задание #3 <a id='3'></a>

In [8]:
with open("subm/w01s02.txt", 'w+') as f:
    f.write(str(len(posfeats) / len(feedbacks)))

print("1 class portion is: {}%".format(int(len(posfeats) / len(feedbacks) * 100)))

1 class portion is: 50%


### Задание #4 <a id='4'></a>

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vect = CountVectorizer()
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
for indx, elm in enumerate(feedbacks):
    feedbacks[indx] = " ".join(elm)
    
len(feedbacks)

2000

In [12]:
vect_features = vect.fit_transform(feedbacks)

In [13]:
%%time
result = vect_features.toarray().shape[1]

with open("subm/w01s03.txt", 'w+') as f:
    f.write(str(result))

print(result)

39659
CPU times: user 20 ms, sys: 504 ms, total: 524 ms
Wall time: 599 ms


### Задание #5 <a id='5'></a>

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(vect_features, feedbacks, test_size=.2, random_state=1)

In [16]:
log_reg = LogisticRegression(random_state=1)

In [17]:
pipe = Pipeline(steps = [
    ('vectorizer', vect),
    ('logistic', log_reg)
])
pipe

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...nalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [18]:
score = model_selection.cross_val_score(pipe, feedbacks, classes)
score

array([ 0.81437126,  0.84684685,  0.84684685])

In [19]:
result = np.mean(score)

with open("subm/w01s04.txt", 'w+') as f:
    f.write(str(result))

print(result)

0.836021650393


### Задание #6 <a id='6'></a>

In [20]:
score = model_selection.cross_val_score(pipe, feedbacks, classes, scoring="roc_auc")
score

array([ 0.9006239 ,  0.91283175,  0.91887383])

In [21]:
result = np.mean(score)

with open("subm/w01s05.txt", 'w+') as f:
    f.write(str(result))

print(result)

0.910776493783


### Задание #7 <a id='7'></a>

In [22]:
lg = LogisticRegression()

In [ ]:
full_lg = lg.fit(vect_features, feedbacks)

In [24]:
coef = full_lg.coef_[0]

In [25]:
values = [vect.get_feature_names()[list(coef).index(i)] for i in sorted(coef, reverse=True)[:5]]
# neg = [vect.get_feature_names()[list(coef).index(i)] for i in sorted(coef, reverse=True)[-5:]]

In [28]:
res = " ".join([values[0], values[1]])

with open("subm/w01s06.txt", 'w+') as f:
    f.write(res)

print(res)

fun great
